# Init

In [1]:
from IPython.display import display
from typing import List,Dict
import os
import sys
import psycopg2
from psycopg2.extras import NamedTupleCursor
os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [2]:
from data_manager.database_manager import db_fetchone,db_no_fetch,db_fetchall
from core.utils.log import log_info,log_error
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        log_info('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchone().version
                log_info(f"PostgreSQL database version: {db_version}")
                log_info(f"PostgreSQL connection status: {conn.info.status}")
                log_info(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        log_error(error)
        conn = None


In [3]:
conn=init_connection(dsn)

2021-07-29 11:34:02.291 INFO    root: Connecting to the PostgreSQL database...
2021-07-29 11:34:02.316 INFO    root: PostgreSQL database version: PostgreSQL 13.3 (Ubuntu 13.3-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit
2021-07-29 11:34:02.318 INFO    root: PostgreSQL connection status: 0
2021-07-29 11:34:02.319 INFO    root: You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'.


# 28/7/2021

In [16]:
from enum import IntEnum
class DeploymentType(IntEnum):
    Image_Classification = 1
    OD = 2
    Instance = 3
    Semantic = 4

    def __str__(self):
        return self.name

    @classmethod
    def from_string(cls, s):
        try:
            return DeploymentType[s]
        except KeyError:
            raise ValueError()

In [17]:
format=DeploymentType.from_string('OD')
format == DeploymentType.OD

True

In [ ]:
return_all=db_fetchone(query_dt_sql,conn)
return_all

In [71]:
for i in return_all:
    display(i)

TypeError: 'NoneType' object is not iterable

In [55]:
y=[i.Model_Path for i in return_all if i.Name=='[TF] SSD MobileNet V2 FPNLite 320x320'][0]
y

'./pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'

In [56]:
model_list=[model.Name for model in return_all]

In [61]:
model_selected='[TF] SSD MobileNet V2 FPNLite 320x320'
model_id=return_all[model_list.index("[TF] SSD MobileNet V2 FPNLite 320x320")].ID
model_id

1

In [12]:
from psycopg2 import sql

In [44]:
query_dt_sql=sql.SQL("""SELECT
                m.id AS "ID",
                m.name AS "Name",
                f.name AS "Framework",
                m.model_path AS "Model Path"
            FROM
                {} m
                LEFT JOIN public.framework f ON f.id = m.framework_id
                where m.deployment_id = (SELECT id from public.deployment_type where name = %s);""").format(sql.Identifier('public','models'))



In [45]:

with conn.cursor() as cur:
    print(cur.mogrify(query_dt_sql,["Object Detection with Bounding Boxes"]))

b'SELECT\n                m.id AS "ID",\n                m.name AS "Name",\n                f.name AS "Framework",\n                m.model_path AS "Model Path"\n            FROM\n                "public"."models" m\n                LEFT JOIN public.framework f ON f.id = m.framework_id\n                where m.deployment_id = (SELECT id from public.deployment_type where name = \'Object Detection with Bounding Boxes\');'


In [46]:
db_fetchall(query_dt_sql,conn,["Object Detection with Bounding Boxes"])

[Record(ID=2, Name='ssd300 retinanet merdeka model', Framework='TensorFlow', Model_Path=None)]

In [50]:
one,two= [x for x in "public.models".split('.')]

In [51]:
one

'public'

In [60]:
def check_if_exists(table: str, column_name: str, condition, conn):
    # Separate schema and tablename from 'table'
    schema, tablename = [i for i in table.split('.')]
    check_if_exists_SQL = sql.SQL("""
                        SELECT
                            EXISTS (
                                SELECT
                                    *
                                FROM
                                    {}
                                WHERE
                                    {} = %s);
                            """).format(sql.Identifier(schema, tablename), sql.Identifier(column_name))
    check_if_exists_vars = [ condition]
    exist_flag = db_fetchone(check_if_exists_SQL, conn, check_if_exists_vars)

    return exist_flag

In [62]:
check_if_exists("public.roles","id",1,conn).exists

True

In [65]:
class Task():
    
        

    # TODO: check if current image exists as a 'task' in DB
    @staticmethod
    def check_if_task_exists(table: str, column_name: str, condition, conn=conn):
        """SELECT
                EXISTS (
                    SELECT
                        *
                    FROM
                        {schema,tablename} --from 'table'
                    WHERE
                        {column_name} = %s); -- %s=condition
                """
        return check_if_exists(table, column_name, condition, conn).exists

In [67]:
Task.check_if_task_exists("public.roles","id",1,conn)

True

# 29/7/2021

In [25]:
from PIL import Image
from io import BytesIO
from base64 import b64encode
from mimetypes import guess_type

In [31]:
image_path=Path("/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/dataset/My-Third-Dataset/IMG_20210315_184229.jpg")

In [37]:
with Image.open(image_path) as img:
    log_info("Loading image")
    
    img_byte=BytesIO()
    log_info("Saving image")
    img.save(img_byte,format=img.format)
    log_info("Done Saving")
    log_info("Encoding")
    bb=img_byte.getvalue()
    
    b64code=b64encode(bb).decode('utf-8')
    log_info("Done encoding")
    mime=guess_type(img.filename)[0]
    image_name=image_path.name
    log_info(f"{image_name} ; {mime}")
    data_url=f"data:{mime};base64{b64code}"
    log_info(f"Data url generated")
    
    

2021-07-29 12:09:46.923 INFO    root: Loading image
2021-07-29 12:09:46.924 INFO    root: Saving image
2021-07-29 12:09:48.130 INFO    root: Done Saving
2021-07-29 12:09:48.131 INFO    root: Encoding
2021-07-29 12:09:48.142 INFO    root: Done encoding
2021-07-29 12:09:48.143 INFO    root: IMG_20210315_184229.jpg ; image/jpeg
2021-07-29 12:09:48.145 INFO    root: Data url generated


In [38]:
from glob import iglob

In [45]:
path="/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/dataset/My-Third-Dataset"
path=path+"/*"
path
dataset_list={}
image_list={}

'/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/dataset/My-Third-Dataset/*'

In [46]:

for image in iglob(path):
    print(Path(image).name)
    image_list=

IMG_20210315_184229.jpg
IMG_20210315_184240.jpg
IMG_20210315_184238.jpg
IMG_20210315_184236.jpg
